In [ ]:
# Libraries
import os
import pandas as pd
from scipy.stats import zscore
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
user = os.getenv('USERPROFILE')
data_path = os.path.join(
    user, 'OneDrive - National University of Singapore\EBAC\Year 1 Semester 1\Project\Data')
os.chdir(data_path)
pd.set_option('max_columns', None)  

In [2]:
retail_df = pd.read_csv('Retail_Week_NUS.csv', parse_dates=[0])
retail_df.head()

,DATE,CUSTNAME,MATERIAL,BASEUOM,MSTAE,H1,H2,H3,QTY_SOLD,CLUSTER
0,2018-06-25,Store 4,10091,PAC,AC,Dairy,Cream Compound,Whipping,5.0,B
1,2018-06-25,Store 4,10550,PAC,AC,"Nuts, Seeds & Beans",Nuts,Pumpkin Kernel,3.0,B
2,2018-06-25,Store 4,11485,PAC,AC,"Flour, Grain & Flakes",Premix,Others,1.0,B
3,2018-06-25,Store 4,1201,PAC,AC,Bakery,Functional,Starches,1.0,B
4,2018-06-25,Store 4,12085,PAC,AC,Grocery,Seasoning,Herbs & Spices,1.0,B


# Data Cleaning

**We only look at active skus and remove General and Service in H1**

In [3]:
retail_df = retail_df.loc[(retail_df['MSTAE'] == 'AC') & (
    (retail_df['H1'] != 'General') & (retail_df['H1'] != 'Service'))]

**Create Year and Week**

In [4]:
retail_df['YEAR'] = retail_df['DATE'].dt.year
retail_df['MONTH'] = retail_df['DATE'].dt.month
retail_df['WEEK'] = retail_df['DATE'].dt.strftime('%W')
print(retail_df.shape)
retail_df.head()

(1391820, 13)


,DATE,CUSTNAME,MATERIAL,BASEUOM,MSTAE,H1,H2,H3,QTY_SOLD,CLUSTER,YEAR,MONTH,WEEK
0,2018-06-25,Store 4,10091,PAC,AC,Dairy,Cream Compound,Whipping,5.0,B,2018,6,26
1,2018-06-25,Store 4,10550,PAC,AC,"Nuts, Seeds & Beans",Nuts,Pumpkin Kernel,3.0,B,2018,6,26
2,2018-06-25,Store 4,11485,PAC,AC,"Flour, Grain & Flakes",Premix,Others,1.0,B,2018,6,26
3,2018-06-25,Store 4,1201,PAC,AC,Bakery,Functional,Starches,1.0,B,2018,6,26
4,2018-06-25,Store 4,12085,PAC,AC,Grocery,Seasoning,Herbs & Spices,1.0,B,2018,6,26


**Insert weeks with no sales (Put 0)<br>Did not put currently as it affects standard deviation**

In [ ]:
retail_group = retail_df.groupby(['YEAR', 'WEEK', 'MATERIAL'])[
    'QTY_SOLD'].sum().reset_index()

In [ ]:
retail_group.head()

In [ ]:
# retail_group = retail_df.pivot_table(index=['YEAR', 'WEEK'], columns=[
#     'MATERIAL'], values='QTY_SOLD', fill_value=0).reset_index()

In [ ]:
# retail_group = retail_group.melt(
#     id_vars=['YEAR', 'WEEK'], value_vars=retail_group.columns[2:], value_name='QTY_SOLD')

In [ ]:
# print(retail_group.shape)
# retail_group.head()

# Seasonal Products

**Sales of items affected by**
- Economy at times
- Nature of Item (Seasonal or Non-seasonal)
- Promotions or Discounts
- Competition from a rival
- Competition from a substitute product
- Special events like Super Bowl, Thanksgiving, New Year, etc varying from places to places

In [ ]:
avg_df = retail_group.groupby(['MATERIAL', 'YEAR'])[
    'QTY_SOLD'].mean().reset_index()
avg_df = avg_df.rename(columns={'QTY_SOLD': 'AVG_QTY'})
avg_df.head()

In [ ]:
retail_merged = retail_group.merge(avg_df, how='left', on=['YEAR', 'MATERIAL'])
retail_merged.head()

**Create seasonal index**

In [ ]:
retail_merged['SEASONAL_INDEX'] = retail_merged['QTY_SOLD'] / \
    retail_merged['AVG_QTY']

In [ ]:
std_df = retail_merged.groupby(['MATERIAL', 'YEAR'])[
    'SEASONAL_INDEX'].std().reset_index()
std_df = std_df.rename(columns={'SEASONAL_INDEX': 'STD'})

In [ ]:
retail_merged = retail_merged.merge(
    std_df, how='left', on=['MATERIAL', 'YEAR'])

**Remove materials that are not sold, it may be cause of lack of inventory**

In [ ]:
retail_merged = retail_merged.dropna(subset=['STD'])

retail_merged = retail_merged.groupby(['YEAR', 'MATERIAL'])['STD'].mean(
).to_frame().sort_values(by=['YEAR', 'STD'], ascending=False)

In [ ]:
high_std = retail_merged.loc[retail_merged['STD'] >= 2]
low_std = retail_merged.loc[retail_merged['STD'] < 1]

**Find common seasonality items for 2019 and 2020**

In [ ]:
s_products = high_std.loc[2019].merge(
    high_std.loc[2020], left_index=True, right_index=True)
non_s_products = low_std.loc[2019].merge(
    low_std.loc[2020], left_index=True, right_index=True)

In [ ]:
s_products

In [ ]:
def plot(mat, year):
    df = retail_df.loc[(retail_df['MATERIAL'] == mat)
                       & (retail_df['YEAR'] == year)]
    df.groupby(['DATE'])['QTY_SOLD'].sum().plot()

In [ ]:
plot(225, 2018)

In [ ]:
test = retail_df.loc[retail_df['MATERIAL'] == 52289]

In [ ]:
test.groupby(['DATE'])['QTY_SOLD'].sum().plot()

# Regression

In [5]:
retail_df['DATE'] = pd.to_datetime(retail_df['DATE']).dt.date

In [6]:
retail_df['DATE_LY'] = retail_df['DATE'] - relativedelta(years=1)
retail_df['DATE_LM'] = retail_df['DATE'] - relativedelta(months=1)
retail_df['DATE_LW'] = retail_df['DATE'] - relativedelta(weeks=1)

**Change to datetime**

In [7]:
retail_df['DATE_LY'] = retail_df['DATE_LY'].apply(pd.to_datetime)
retail_df['DATE_LM'] = retail_df['DATE_LM'].apply(pd.to_datetime)
retail_df['DATE_LW'] = retail_df['DATE_LW'].apply(pd.to_datetime)

In [ ]:
retail_df

In [ ]:
retail_df['YEAR_LY'] = retail_df['DATE_LY'].dt.year
retail_df['MONTH_LY'] = retail_df['DATE_LY'].dt.month
retail_df['WEEK_LY'] = retail_df['DATE_LY'].dt.strftime('%W')

retail_df['YEAR_LM'] = retail_df['DATE_LM'].dt.year
retail_df['MONTH_LM'] = retail_df['DATE_LM'].dt.month
retail_df['WEEK_LM'] = retail_df['DATE_LM'].dt.strftime('%W')

retail_df['YEAR_LW'] = retail_df['DATE_LW'].dt.year
retail_df['MONTH_LW'] = retail_df['DATE_LW'].dt.month
retail_df['WEEK_LW'] = retail_df['DATE_LW'].dt.strftime('%W')

In [ ]:
test.columns

In [ ]:
# Get slice of dataframe first
retail = retail_df[['YEAR_LY', 'MONTH_LY',
                    'WEEK_LY', 'CUSTNAME', 'MATERIAL', 'QTY_SOLD']]

In [ ]:
retail_df.loc[retail_df['DATE'] == '2020-04-05']

In [ ]:
test = retail_df.merge(retail, how='left', left_on=['YEAR', 'MONTH', 'WEEK','CUSTNAME','MATERIAL'],
                       right_on=['YEAR_LY', 'MONTH_LY', 'WEEK_LY','CUSTNAME','MATERIAL'])

In [ ]:
test[~test['QTY_SOLD_y'].isnull()]

In [ ]:
retail_df.dtypes

In [ ]:
retail.dtypes

In [ ]:
test['QTY_SOLDQTY_LY'].unique()

In [ ]:
retail_df.loc[retail_df['DATE_LY'] == retail_df['DATE']]